<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import copy
import numpy as np
import random
import math

moves=[[-1,0], [-1,-1], [0, -1], [1,-1], [1,0], [1, 1], [0, 1], [-1, 1]]
directions = [1,2,3,4,5,6,7,8];

def find_cost(old_move, new_move):
  if abs(old_move - new_move) == 1 or abs(old_move - new_move) == 7:
    return 1
  elif abs(old_move - new_move) == 2 or abs(old_move - new_move) == 6:
    return 2
  elif abs(old_move - new_move) == 3 or abs(old_move - new_move) == 5:
   return 3
  elif abs(old_move - new_move) == 0:
    return 0
  else:
    return 4

def find_way(end):
  if end[0] == 0 and end[1] == 0:
    return random.randint(5,7)
  elif end[0] == 8 and end[1] == 8:
    return random.randint(1,3)
  elif end[0] == 8 and end[1] == 0:
    return random.choice([1,7,8])
  elif end[0] == 0 and end[1] == 8:
    return random.randint(3,5)
  elif end[0] == 8:
    return random.choice([3,2,1,7,8])
  elif end[0] == 0:
    return random.choice([3,4,5,6,7])
  elif end[1] == 8:
    return random.choice([1,2,3,4,5])
  elif end[1] == 0:
    return random.choice([5,6,7,8,1])
  else:
    return random.randint(1,8)

def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  route_matrix_list = []
  end=[0,0]
  for route in coming_routes:  #population 5000
    flag=0
    cost = 0            
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end[0]=start[0]
    end[1]=start[1]
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:  
          end = [end[0] - add_move[0], end[1] - add_move[1]]
          route[i]=find_way(end)
          add_move =moves[route[i]-1]
          end = [end[0] + add_move[0], end[1] + add_move[1]]
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1  
    route_matrix_list.append(matrix)
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    
    fitness_list.append(find_fitness(start, end, cost, area))
  return return_routes,fitness_list,route_matrix_list



def group(routes,numberOfDrone,fitness_list, matrix_list):
    i=0
    fitness=[]
    routes1_list=[]
    routes2_list=[]

    while i<(len(routes)):
        sum=similarity(matrix_list[i],matrix_list[i+1])
        fitness.append(find_fitness2(fitness_list[i], fitness_list[i+1], sum))
        routes1_list.append(routes[i])
        routes2_list.append(routes[i+1])
        #print(sum)
        i=i+2
    zipped = list(zip(routes1_list, routes2_list,fitness))   
    sorted_routes = sorted(zipped, key = lambda x: x[2])
    unzipped_list = list(zip(*sorted_routes))
    return unzipped_list

def create_first_route(start,numberOfDrone,length):
  fitness_list = []
  routes = []
  matrix_list = []
  
  for i in range (population*numberOfDrone):
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1;
    end = copy.deepcopy(start)
    route = []
    cost = 0
    route.append(1)
    while len(route) <= length:
      route.append(find_way(end))
      cost += find_cost(route[-2], route[-1])
      add_move = moves[route[-1]-1]
      end[0] += add_move[0]
      end[1] += add_move[1]
      matrix[end[0], end[1]] = 1
    
    area = len(np.argwhere(matrix == 1))
    routes.append(route)
    matrix_list.append(matrix)
    fitness_list.append(find_fitness(start, end, cost, area))
    
  return routes, fitness_list,matrix_list


def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = (area) - cost - (distance)
  return fitness

def cross_over(routes,length):
  new_generation = []
  length=(length/2)+1
  count=0
  for i in range (len(routes)):

    route1 = random.choice(routes)
    route2 = random.choice(routes)
    #new_route1 = route1[:1]+route1[1:21] + route2[21:41] + route1[41:61] + route2[61:]
    #new_route2 = route2[:1]+route2[1:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_route1 = route1[:1]+route1[1:int(length)] + route2[int(length):] 
    new_route2 = route2[:1]+route2[1:int(length)] + route1[int(length):]      

    new_generation.append(new_route1)
    new_generation.append(new_route2)

  return new_generation

def mutation(routes, mutation_ratio, length, population):
  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]        #route sayısı çok azaldığı için hata alıyoruz
    for i in range (3):
      index2 = random.randint(1,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes

def similarity(mat1,mat2):
  sum = 0
  print(mat1)
  print(mat2)
  for row1,row2 in zip(mat1,mat2):
    for elem1,elem2 in zip(row1,row2):
      if (elem1 == elem2) & (elem2 == 1):
        sum += 1
  return sum

def find_fitness2(fitness1, fitness2, sum):
  return fitness1 + fitness2 - 8*sum


size = 9
population = 5000
numberOfDrone=2
selected = int(population/2)
generation = 200
length = ((size*size-1)/numberOfDrone)
mutation_ratio = 0.01
matrix_list=[]
matrix_list2=[]
start = [5,5]
end = copy.deepcopy(start)
routes, fitness_list ,matrix_list = create_first_route(start,numberOfDrone,length)

if numberOfDrone >1:
  unzipped_list=group(routes,numberOfDrone,fitness_list,matrix_list)
  print("dslsdkdskl", len(unzipped_list[0]))
  new_route_list=[]
  new_fitness_list=[]
  new_matrix_list=[]
  for i in range (generation-1):
      print("i",i)
      for j in range(numberOfDrone) :
          print("unzipped", len(unzipped_list[j]))
          print("selected",selected)
          cross_generation = cross_over((unzipped_list[j])[selected:], length)
          mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population) 
          new_routes, new_fitness, new_matrix = try_routes(mutated_generation, start, size)
          print(new_routes)
          new_route_list += new_routes
          print(new_route_list[0])
          new_fitness_list += new_fitness
          new_matrix_list += new_matrix
      break;
  print("len", len(new_route_list), len(new_fitness_list), len(new_matrix_list))
  unzipped_list=group(new_route_list,numberOfDrone,new_fitness_list,new_matrix_list)
  #print(i)
else:
  zipped = list(zip(routes, fitness_list))
  sorted_routes = sorted(zipped, key = lambda x: x[1])
  unzipped_list = list(zip(*sorted_routes))
  new_generation = unzipped_list[0]

  for i in range (generation-1):
    cross_generation = cross_over(new_generation[selected:],length)   
    #print("new gen:",len(cross_generation))
    mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population)
    #print("muntanted gen:",len(mutated_generation))  
    new_routes, new_fitness, matrix_list= try_routes(mutated_generation, start, size)
    #print("new route_list length:", len(new_routes))
    zipped = list(zip(new_routes, new_fitness))
    sorted_routes = sorted(zipped, key = lambda x: x[1])
    unzipped_list = list(zip(*sorted_routes))
    new_generation = unzipped_list[0]
    print("döngü",i+1)
print("bitti")

Streaming output truncated to the last 5000 lines.
 [0. 0. 0. 1. 1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0

In [ ]:
print((unzipped_list[0][:10]))
print((unzipped_list[1][:10]))
#print(str(unzipped_list[2][5]))
#print(len(unzipped_list[1]))

In [73]:
end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix1 = np.zeros((size,size))
for i in range (int(length)):
  add_move =moves[unzipped_list[0][0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix1[end[0], end[1]] = 1
print(end)
area1 = len(np.argwhere(matrix1 == 1))
print(area1)

matrix2 = np.zeros((size,size))
for i in range (int(length)):
  add_move =moves[unzipped_list[1][0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix2[end[0], end[1]] = 1
print(end)
area2 = len(np.argwhere(matrix2 == 1))
print(area2)
similarity(matrix1,matrix2)


start: [5, 5]
end [5, 5]
end: [4, 5]
end: [5, 4]
end: [4, 3]
end: [3, 3]
end: [4, 3]
end: [3, 2]
end: [3, 3]
end: [2, 2]
end: [3, 1]
end: [3, 2]
end: [2, 2]
end: [3, 3]
end: [2, 2]
end: [2, 1]
end: [3, 0]
end: [3, 1]
end: [2, 2]
end: [3, 3]
end: [3, 4]
end: [2, 3]
end: [3, 3]
end: [4, 3]
end: [3, 4]
end: [2, 4]
end: [3, 4]
end: [2, 5]
end: [3, 6]
end: [4, 6]
end: [5, 6]
end: [6, 7]
end: [7, 7]
end: [7, 8]
end: [7, 7]
end: [8, 6]
end: [8, 5]
end: [8, 6]
end: [7, 6]
end: [8, 7]
end: [7, 8]
end: [7, 7]
[7, 7]
23
end: [6, 7]
end: [7, 6]
end: [6, 5]
end: [5, 5]
end: [6, 5]
end: [5, 4]
end: [5, 5]
end: [4, 4]
end: [5, 3]
end: [5, 4]
end: [4, 4]
end: [5, 5]
end: [4, 4]
end: [4, 3]
end: [5, 2]
end: [5, 3]
end: [4, 4]
end: [5, 5]
end: [5, 6]
end: [4, 5]
end: [5, 5]
end: [6, 5]
end: [5, 6]
end: [4, 6]
end: [5, 6]
end: [4, 7]
end: [5, 8]
end: [6, 8]
end: [7, 8]
end: [8, 9]


IndexError: ignored